# Chapter 5: companion notebook

This notebook contains some of the code used in Chapter 5 of the book. It is not intended to be a complete record of all the code used in the chapter, but rather to show some of the key steps in the analysis.

### Cifar10 classification using CNNs

Complete implementation of a simple CNN for classifying images from the Cifar10 dataset. 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define a CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the data
dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transforms.ToTensor())

trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=4,
                                            shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transforms.ToTensor()) 
            
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                          shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the model
net = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

def train_one_epoch(net, trainloader, criterion, optimizer):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # to GPU
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    return running_loss / len(trainloader)

def evaluate(net, valloader, criterion):
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(valloader):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(valloader), correct / total

# Train the model
net.to(device)

for epoch in range(10):  # loop over the dataset multiple times

    train_loss = train_one_epoch(net, trainloader, criterion, optimizer)
    val_loss, val_acc = evaluate(net, valloader, criterion)
    print(f"Epoch {epoch} - Train loss: {train_loss:.3f} - Val loss: {val_loss:.3f} - Val acc: {val_acc:.3f}")

print('Finished Training')

# Evaluate the model on the test set
test_loss, test_acc = evaluate(net, testloader, criterion)
print(f"Test loss: {test_loss:.3f} - Test acc: {test_acc:.3f}")

100%|██████████| 170498071/170498071 [00:23<00:00, 7396215.40it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 0 - Train loss: 2.002 - Val loss: 1.704 - Val acc: 0.372


...

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 4 - Train loss: 1.284 - Val loss: 1.350 - Val acc: 0.516


  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Cifar10 classification using pre-trained ViT

Complete implementation of a Vision Transformer (ViT) for classifying images from the Cifar10 dataset. The pre-trained model is available from the [transformer model hub](https://huggingface.co/google/vit-base-patch16-224) with the tag `google/vit-base-patch16-224`.

In [10]:
#free up memory
del net
torch.cuda.empty_cache()

NameError: name 'net' is not defined

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm
from transformers import ViTFeatureExtractor, ViTForImageClassification

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ViTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
        # CiFAR images are 32x32, ViT requires 224x224
        self.resize = transforms.Resize((224, 224))

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = self.resize(image)
        image = self.feature_extractor(images=image, return_tensors="pt")["pixel_values"]
        return {
            "pixel_values": image.squeeze(),
            "labels": label
        }

    def __len__(self):
        return len(self.dataset)

# Load the data
def get_cifar_dataloaders():
    dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transforms.ToTensor())

    trainset, valset = torch.utils.data.random_split(dataset, [40000, 10000])
    trainset = ViTDataset(trainset)
    valset = ViTDataset(valset)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                            shuffle=True, num_workers=2)

    valloader = torch.utils.data.DataLoader(valset, batch_size=16,
                                                shuffle=False, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transforms.ToTensor())
    testset = ViTDataset(testset)
            
    testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                            shuffle=False, num_workers=2)
    return trainloader, valloader, testloader

trainloader, valloader, testloader = get_cifar_dataloaders()

# Define the model
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=10, ignore_mismatched_sizes=True)
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_one_epoch(model, trainloader, criterion, optimizer):
    running_loss = 0.0
    for i, batch in enumerate(tqdm(trainloader)):
        batch = {k: v.to(device) for k, v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(batch["pixel_values"])
        logits = outputs.logits
        loss = criterion(logits, batch["labels"])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    return running_loss / len(trainloader)

def evaluate(model, valloader, criterion):
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(valloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch["pixel_values"])
            labels = batch["labels"]
            logits = outputs.logits
            loss = criterion(logits, labels)
            running_loss += loss.item()
            predicted = torch.argmax(logits, dim=-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(valloader), correct / total

# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    train_loss = train_one_epoch(model, trainloader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, valloader, criterion)
    print(f"Epoch {epoch} - Train loss: {train_loss:.3f} - Val loss: {val_loss:.3f} - Val acc: {val_acc:.3f}")

print('Finished Training')

# Evaluate the model on the test set
test_loss, test_acc = evaluate(model, testloader, criterion)
print(f"Test loss: {test_loss:.3f} - Test acc: {test_acc:.3f}")

Files already downloaded and verified


/home/mlaquatra/miniconda3/envs/audio/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Files already downloaded and verified


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2500 [00:00<?, ?it/s]

/home/mlaquatra/miniconda3/envs/audio/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
/home/mlaquatra/miniconda3/envs/audio/lib/python3.10/site-packages/torchvision/trans

KeyboardInterrupt: 

## Keyword spotting using a pre-trained Wav2Vec2 model

Implementaion of a Wav2Vec2 model for keyword spotting. The pre-trained model is available from the [transformer model hub](https://huggingface.co/facebook/wav2vec2-base-960h) with the tag `facebook/wav2vec2-base-960h`.

The dataset used is the [superb](https://huggingface.co/datasets/superb) dataset from the [Hugging Face datasets](https://huggingface.co/datasets) library.

In [1]:
# load dataset
from datasets import load_dataset
train_dataset = load_dataset("superb", "ks", split="train")
val_dataset = load_dataset("superb", "ks", split="validation")
test_dataset = load_dataset("superb", "ks", split="test")

print(train_dataset[0])
print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of validation examples: {len(val_dataset)}")
print(f"Number of testing examples: {len(test_dataset)}")


Found cached dataset superb (/home/mlaquatra/.cache/huggingface/datasets/superb/ks/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e)
Found cached dataset superb (/home/mlaquatra/.cache/huggingface/datasets/superb/ks/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e)
Found cached dataset superb (/home/mlaquatra/.cache/huggingface/datasets/superb/ks/1.9.0/b8183f71eabe8c559d7f3f528ab37a6a21ad1ee088fd3423574cecad8b3ec67e)


{'file': '/home/mlaquatra/.cache/huggingface/datasets/downloads/extracted/190330ac5b845198a860d440f885500397d47675495993cca8da2b1b9f36affb/_background_noise_/doing_the_dishes.wav', 'audio': {'path': '/home/mlaquatra/.cache/huggingface/datasets/downloads/extracted/190330ac5b845198a860d440f885500397d47675495993cca8da2b1b9f36affb/_background_noise_/doing_the_dishes.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00592041,
       -0.00405884, -0.00253296], dtype=float32), 'sampling_rate': 16000}, 'label': 10}
Number of training examples: 51094
Number of validation examples: 6798
Number of testing examples: 3081


In [2]:
# Define the model
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor
unique_labels = set([example["label"] for example in train_dataset])
num_labels = len(unique_labels)
model = AutoModelForAudioClassification.from_pretrained("microsoft/wavlm-base-plus", num_labels=num_labels)
print(f"Initalized model with {num_labels} labels")

Some weights of WavLMForSequenceClassification were not initialized from the model checkpoint at microsoft/wavlm-base-plus and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight', 'classifier.bias', 'projector.bias', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initalized model with 12 labels


In [3]:
# implement the dataset class
import torch

class KSDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/wavlm-base-plus")
        self.max_length_in_seconds = 2

    def __getitem__(self, idx):
        audio_array = self.dataset[idx]["audio"]["array"]
        label = self.dataset[idx]["label"]
        audio = self.feature_extractor(
            audio_array,
            sampling_rate=16000,
            max_length=self.max_length_in_seconds * 16000,
            padding="max_length",
            return_tensors="pt",
            truncation=True
        ).input_values
        return {
            "input_values": audio.squeeze(),
            "labels": label
        }

    def __len__(self):
        return len(self.dataset)
    
train_ds = KSDataset(train_dataset)
val_ds = KSDataset(val_dataset)
test_ds = KSDataset(test_dataset)

In [7]:
from torch import nn, optim
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# data loaders
trainloader = torch.utils.data.DataLoader(train_ds, batch_size=16,
                                            shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_ds, batch_size=16,
                                            shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(test_ds, batch_size=16,
                                            shuffle=False, num_workers=2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
lamda_fn = lambda epoch: 0.95 ** epoch
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lamda_fn)

def train_one_epoch(model, trainloader, criterion, optimizer):
    running_loss = 0.0
    for i, batch in enumerate(tqdm(trainloader)):
        batch = {k: v.to(device) for k, v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(batch["input_values"])
        logits = outputs.logits
        loss = criterion(logits, batch["labels"])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    return running_loss / len(trainloader)

def evaluate(model, valloader, criterion):
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(valloader):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch["input_values"])
            labels = batch["labels"]
            logits = outputs.logits
            loss = criterion(logits, labels)
            running_loss += loss.item()
            predicted = torch.argmax(logits, dim=-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(valloader), correct / total

# Train the model
model.to(device)
best_model = None
best_acc = 0.0
for epoch in range(10):  # loop over the dataset multiple times
    train_loss = train_one_epoch(model, trainloader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, valloader, criterion)
    if val_acc > best_acc:
        best_acc = val_acc
        best_model = model.state_dict()
    print(f"Epoch {epoch} - Train loss: {train_loss:.3f} - Val loss: {val_loss:.3f} - Val acc: {val_acc:.3f}")
    scheduler.step()
    
print('Finished Training')

# load the best model
model.load_state_dict(best_model)

# Evaluate the model on the test set
test_loss, test_acc = evaluate(model, testloader, criterion)
print(f"Test loss: {test_loss:.3f} - Test acc: {test_acc:.3f}")


  0%|          | 0/3194 [00:00<?, ?it/s]

2024-01-29 19:13:32.962025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 19:13:32.963358: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 19:13:33.029174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 19:13:33.030482: I tensorflow/core/platform/cpu_feature_guard.cc:182]

  0%|          | 0/425 [00:00<?, ?it/s]

2024-01-29 19:32:43.575256: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 19:32:43.578064: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 19:32:43.631144: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 19:32:43.633569: I tensorflow/core/platform/cpu_feature_guard.cc:182]

Epoch 0 - Train loss: 1.502 - Val loss: 1.540 - Val acc: 0.621


...

  0%|          | 0/425 [00:00<?, ?it/s]

2024-01-29 20:48:16.201882: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 20:48:16.201882: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 20:48:16.255685: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 20:48:16.255685: I tensorflow/core/platform/cpu_feature_guard.cc:182]

Epoch 4 - Train loss: 1.494 - Val loss: 1.537 - Val acc: 0.621


  0%|          | 0/3194 [00:00<?, ?it/s]

2024-01-29 20:49:00.304797: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 20:49:00.312411: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-29 20:49:00.357977: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 20:49:00.364405: I tensorflow/core/platform/cpu_feature_guard.cc:182]